In [1]:
import sqlite3
import torch

import sklearn.metrics
import h5py
import numpy as np
# import nn_learner
import matplotlib.path as mpltPath
# from glueCodeTypes import SolverCode, BGKInputs, BGKOutputs
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum, IntEnum
import collections
import torch
import sklearn.metrics
from shapely import geometry
from shapely.geometry import Point

import hdfdict
import numpy as np
import netCDF4 as nc
import os
import h5py

In [2]:
# !pip install shapely
# ! pip install hdfdict
# !pip install netCDF4
# !pip install h5py

In [3]:
# path_data='/Users/42d/WEST/SOLEDGEx/'  # HARDCODED 
nR,nZ=500,1000

In [4]:
def getAllGNDData(dbPath, solverCode):
    selString = ""
    if solverCode == SolverCode.BGK:
        selString = "SELECT * FROM SOLEDGE;"
    else:
        raise Exception('Using Unsupported Solver Code')
    sqlDB = sqlite3.connect(dbPath, timeout=45.0)
    sqlCursor = sqlDB.cursor()
    gndResults = []
    for row in sqlCursor.execute(selString):
        # Add row to numpy array
        gndResults.append(row)
    sqlCursor.close()
    sqlDB.close()
    return np.array(gndResults)

def project_parallel_variable_xyz(variable,br,bt,bz):
    btot = np.sqrt(np.multiply(br,br) + np.multiply(bt,bt) + np.multiply(bz,bz))

    
    variable_r = np.multiply(br,variable)/btot
    variable_t = np.multiply(bt,variable)/btot
    variable_z = np.multiply(bz,variable)/btot
    
    variable_r=np.nan_to_num(variable_r)
    variable_t=np.nan_to_num(variable_t)
    variable_z=np.nan_to_num(variable_z)
    
    return variable_r,variable_t,variable_z    


def function(model, r,z):
    
    # Load torch model and return results for (r,z)
    core=np.loadtxt(path_data+'core.csv', unpack=True)

    Inputs = collections.namedtuple('SOLEDGE', 'R Z')
    
    L=[]
    
    core_ = [(core[0][i], core[1][i]) for i in range(len(core[0]))]
    
    mesh=h5py.File(path_data+'mesh.h5','r')
    Rwall=mesh['/wall/R'][...]
    Zwall=mesh['/wall/Z'][...]
    wall = [(Rwall[i], Zwall[i]) for i in range(len(Zwall))]

    for i in r:
        val =[]
        for j in z:
            pts = [(i,j)]
            s=Inputs(i, j)
            inside_core = mpltPath.Path(core_).contains_points(pts)
            inside_wall= mpltPath.Path(wall).contains_points(pts)
            if inside_wall and not inside_core:
                [b_phi,b_r,b_z,T_e,n_e,v_e,t_i,n_i,v_i]=[model(s)[0][i] for i in range(9)]
            else:
                [b_phi,b_r,b_z,T_e,n_e,v_e,t_i,n_i,v_i]=np.zeros(9)
            val.append([b_phi,b_r,b_z,T_e,n_e,v_e,t_i,n_i,v_i])  
        L.append(val)  
    
    return np.asarray(L)


In [5]:
!pwd

/Users/78k/Library/CloudStorage/OneDrive-OakRidgeNationalLaboratory/ORNL-ATUL-MBP/myRepos/SOLEDGE2D


In [6]:
#Torch Model
path='/Users/42d/WEST/Bfield/glueCode_proofOfConcept-master/DB/Skyrme/skyrme_model/'
model=torch.load(path+"data.pt")
fields= function(model, rdak,zdak)

In [7]:
# [b_phi,b_r,b_z,T_e,n_e,v_e,t_i,n_i,v_i]=[fields[:,:,i] for i in range(9)]

NameError: name 'fields' is not defined

In [ ]:
# off_grid_inds = np.where(n_i < 0)
# n_i[off_grid_inds] = 0.0;

# off_grid_inds = np.where(n_e < 0)
# n_e[off_grid_inds] = 0.0;


# off_grid_inds = np.where(T_e < 0)
# t_emin=2.69422414e-05
# T_e[off_grid_inds] = t_emin;

# off_grid_inds = np.where(t_i < 0)
# t_imin=1.71878994e-04
# t_i[off_grid_inds] = t_imin;

In [ ]:
# br=b_r
# bz=b_z
# bt=b_phi

# vi=v_i
# ni=n_i
# V_para=vi

# vel_r,vel_t,vel_z = project_parallel_variable_xyz(vi,br,bt,bz)

In [ ]:
# btot = np.sqrt(np.multiply(br,br) + np.multiply(bt,bt) + np.multiply(bz,bz))

In [10]:
# Create a plasma background file for GITR


outputFileName='profiles'
profiles_filename = str(outputFileName)+".nc"
if os.path.exists(profiles_filename):
    os.remove(profiles_filename)

rootgrp = nc.Dataset(profiles_filename, "w", format="NETCDF4")

nrr = rootgrp.createDimension("nR", nR)
nzz = rootgrp.createDimension("nZ", nZ)

rr = rootgrp.createVariable("gridR", "f8", ("nR"))
zz = rootgrp.createVariable("gridZ", "f8", ("nZ"))
    
nee = rootgrp.createVariable("ne", "f8", ("nZ", "nR"))
tee = rootgrp.createVariable("te", "f8", ("nZ", "nR"))

nii = rootgrp.createVariable("ni", "f8", ("nZ", "nR"))
tii = rootgrp.createVariable("ti", "f8", ("nZ", "nR"))

E_para = rootgrp.createVariable("Epara", "f8", ("nZ", "nR"))
Ett = rootgrp.createVariable("et", "f8", ("nZ", "nR"))
Err = rootgrp.createVariable("er", "f8", ("nZ", "nR"))
Ezz = rootgrp.createVariable("ez", "f8", ("nZ", "nR"))
    
vtt = rootgrp.createVariable("vt", "f8", ("nZ", "nR"))
vrr = rootgrp.createVariable("vr", "f8", ("nZ", "nR"))
vzz = rootgrp.createVariable("vz", "f8", ("nZ", "nR"))
V_para = rootgrp.createVariable("Vpara", "f8", ("nZ", "nR"))


b_tot = rootgrp.createVariable("btot", "f8", ("nZ", "nR"))
brr = rootgrp.createVariable("br", "f8", ("nZ", "nR"))
btt = rootgrp.createVariable("bt", "f8", ("nZ", "nR"))
bzz = rootgrp.createVariable("bz", "f8", ("nZ", "nR"))

teer = rootgrp.createVariable("gradTeR", "f8", ("nZ", "nR"))
teez = rootgrp.createVariable("gradTeZ", "f8", ("nZ", "nR"))
teey = rootgrp.createVariable("gradTeY", "f8", ("nZ", "nR"))

tiir = rootgrp.createVariable("gradTiR", "f8", ("nZ", "nR"))
tiiz = rootgrp.createVariable("gradTiZ", "f8", ("nZ", "nR"))
tiiy = rootgrp.createVariable("gradTiY", "f8", ("nZ", "nR"))


In [11]:
%ls

GT_data.txt                        mesh_raptorX.h5
InterpolationPlasmaProfiles.ipynb  metric_raptorX.h5
ProcessingSolEdgeData.ipynb        plasma_00010.h5
mesh.h5                            profiles.nc
meshEIRENE.h5                      refParam_raptorX.h5


In [12]:
data=nc.Dataset('profiles.nc')

In [16]:
rdak=np.linspace(1.8,3.6,500)
zdak=np.linspace(-1,1,1000)

In [17]:
rr[:] = rdak
zz[:] = zdak
nee[:] = ne

In [19]:
data=nc.Dataset('profiles.nc')
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): nR(500), nZ(1000)
    variables(dimensions): float64 gridR(nR), float64 gridZ(nZ), float64 ne(nZ, nR), float64 te(nZ, nR), float64 ni(nZ, nR), float64 ti(nZ, nR), float64 Epara(nZ, nR), float64 et(nZ, nR), float64 er(nZ, nR), float64 ez(nZ, nR), float64 vt(nZ, nR), float64 vr(nZ, nR), float64 vz(nZ, nR), float64 Vpara(nZ, nR), float64 btot(nZ, nR), float64 br(nZ, nR), float64 bt(nZ, nR), float64 bz(nZ, nR), float64 gradTeR(nZ, nR), float64 gradTeZ(nZ, nR), float64 gradTeY(nZ, nR), float64 gradTiR(nZ, nR), float64 gradTiZ(nZ, nR), float64 gradTiY(nZ, nR)
    groups: 

In [ ]:
rr[:] = rdak
zz[:] = zdak

nee[:] = n_e.T
tee[:] = T_e.T

nii[:] = n_i.T
tii[:] = t_i.T

b_tot[:] = btot.T
brr[:] = br.T
btt[:] = bt.T
bzz[:] = bz.T

V_para[:]= vi.T
vrr[:] = vel_r.T
vzz[:] = vel_z.T
vtt[:] = vel_t.T

Ett[:] = e_t
Err[:] = e_par
Ezz[:] = e_perp

teer[:] = grad_te_r
teez[:] = grad_te_z
teey[:] = grad_te_t

tiir[:] = grad_ti_r
tiiz[:] = grad_ti_z
tiiy[:] = grad_ti_t

rootgrp.close()    

In [ ]:
!pwd

In [ ]:
data=nc.Dataset('profiles.nc')

In [ ]:
!pwd
